In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

# WritingProject

## 长文内容结构

In [2]:
from langchain_chinese import TreeContent
root = TreeContent()

t1 = root.add_item(TreeContent(text="H1"))
t2 = root.add_item(TreeContent(text="H2"))

In [3]:
t2

'20240505.105434.00003.723'

In [4]:
h1 = root.get_item_by_id(t1)
h2_id = h1.add_item(TreeContent(text="my headline 1"))
root.get_item_by_id(h2_id)

TreeContent(id='20240505.105436.00004.159', type='paragraph', is_completed=False, words_advice=500, summarise=None, text='my headline 1', children=[], path=None)

In [7]:
root.all_todos()

[{'id': '20240505.105434.00001.560', 'type': 'outline'},
 {'id': '20240505.105434.00002.734', 'type': 'outline'},
 {'id': '20240505.105436.00004.159', 'type': 'paragraph'},
 {'id': '20240505.105434.00003.723', 'type': 'paragraph'}]

In [8]:
t1 = root.next_todo()
t1

TreeContent(id='20240505.105436.00004.159', type='paragraph', is_completed=False, words_advice=500, summarise=None, text='my headline 1', children=[], path=None)

In [9]:
t1.is_completed=True

In [10]:
t2 = root.next_todo()
t2

TreeContent(id='20240505.105434.00003.723', type='paragraph', is_completed=False, words_advice=500, summarise=None, text='H2', children=[], path=None)

In [11]:
t2.is_completed=True

In [13]:
root.all_todos()

[{'id': '20240505.105434.00001.560', 'type': 'outline'},
 {'id': '20240505.105434.00002.734', 'type': 'outline'}]

## 长文写作智能体

In [24]:
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_zhipu import ChatZhipuAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json
import re

TASK_INSTRUCTIONS = """
你是一名优秀的写手，可以构思写作思路、扩展写作提纲、细化段落内容，

请务必记住：
1. 当你收到新的写作任务，你应当做两种选择，要么输出写作提纲，要么输出细化的写作内容。
2. 你输出的内容规定为最大不超过300字。
3. 如果发现用户要求的字数超出了最大限制规定，你就必须输出写作提纲；反之，你就输出段落内容。
4. 写作提纲中的大纲数量必须大于2，否则就没必要作为写作提纲，直接输出为段落内容即可。

例如：
用户要求字数1500字左右，此时超出了300字的写作限定，你必须输出“写作提纲”，可以分为5个部份，每部份约300字左右；
用户要求字数80字左右，此时符合300字左右的限定，你必须输出为“段落内容”。

如果你决定输出“写作提纲”，就请按照如下格式输出写作大纲：
```json
{
    "类型": "outlines",
    "大纲列表": [
        {"标题名称": "标题名称", "字数要求": 字数要求（int类型）, "内容摘要": 内容摘要},
        {"标题名称": "标题名称", "字数要求": 字数要求（int类型）, "内容摘要": 内容摘要},
        （更多行）
        {"标题名称": "标题名称", "字数要求": 字数要求（int类型）, "内容摘要": 内容摘要}
    ],
    "大纲数量": 与以上列表相符的大纲数量
}
```

如果你决定输出“段落内容”，就请按照如下格式输出：
```json
{
    "类型": "paragraph",
    "字数要求": 字数要求（int类型）,
    "内容": "你的详细输出"
}
```

只输出上述的JSON内容即可，其他不必输出。
"""

def get_input(prompt: str = "\n👤: ") -> str:
    return input(prompt)

class WritingProject(BaseModel):
    """
    写作管理。
    """
    root_content = TreeContent()

    # 游标从根文档开始
    cur_content = root_content

    # 控制参数
    words_per_step = 300
    words_all_limit = 1000

    # - 子任务可以配置为几种处置方案：Skip / Redo / AskMe
    # - Skip 可以跳过处理，采纳原有的结果或暂时不处理
    # - Redo 重新分配 session_id 生成，但用户全部自动回复 OK 
    # - RedoN 未来可支持 Redo N 次，由 LLM 代替用户做最多 N 次评估
    # - AskMe 重新分配 session_id 生成，但要求用户介入评估
    default_task_mode = "AskMe"

    streaming = False

    def next_step(self):
        pass

    def ask_user(self, ai_said: str = "") -> tuple:
        resp = get_input()
        content = ""
        command = ""
    
        if(resp == "quit"):
            command = "quit"
        elif(resp == "ok"):
            content = ai_said
            command = "ok"
        else:
            content = resp
            command = "chat"
    
        return content, command

    def get_chain(self):
        prompt_init = ChatPromptTemplate.from_messages([
            ("system", TASK_INSTRUCTIONS),
            ("user", "{{input}}")
        ], template_format='jinja2')

        prompt_detail = ChatPromptTemplate.from_messages([
            ("system", TASK_INSTRUCTIONS),
            ("assistant", "之前的写作提纲为: {{outlines}}"),
            ("user", "{{input}}。请注意，你现在的写作任务是上面已有提纲的一部份")
        ], template_format='jinja2')

        return (prompt_init | ChatZhipuAI() | JsonOutputParser())

    def run(self):
        # 初始化链
        chain = self.get_chain()
        ai_said = {}

        while True:
            # 用户输入
            user_said, command = self.ask_user(ai_said)

            if command == "quit":
                print("-"*20, "quit")
                break
                
            elif command == "ok":
                print("-"*20, "ok")

                # 生成目录或文件
                if "类型" in ai_said:
                    task_type = ai_said['类型']
                else:
                    print("Error AI Said: ", ai_said)
                    continue

                # 如果确认要生成大纲
                if task_type == "outlines":
                    for item in ai_said['大纲列表']:
                        self.cur_content.add_item(TreeContent(
                            words_advice = item['字数要求'],
                            text = item['标题名称'],
                            summarise = item['内容摘要'],
                            is_completed = False,
                        ))
                    # 生成子任务后，提纲自身的任务就算完成了
                    self.cur_content.is_completed = True
                elif task_type == "paragraph":
                    self.cur_content.text = item['内容']
                    self.cur_content.words_advice = item['字数要求'],
                    self.cur_content.is_completed = True
                else:
                    print("Error JSON: ", ai_said)
                    continue

                # 获取下一个任务的计划
                next_todo = self.root_content.next_todo()
                
                if next_todo:
                    # 如果下一个任务存在，继续转移到新的任务主题
                    self.cur_content = next_todo
                    user_said = f'请帮我扩写{next_todo.text}，内容摘要为{next_todo.summarise}, 字数不少于{next_todo.words_advice}'
                    chain = self.get_chain()
                else:
                    # 如果没有下一个任务，就结束
                    print("-"*20, "task complete!")
                    print(self.root_content.all_todos())
                    break;
                
            if len(user_said) > 0:
                # 如果根文档内容为空，就采纳用户输入的第一句不为特定
                if self.root_content.text == None:
                    self.root_content.text = user_said
                    
                # AI推理
                if self.streaming:
                    for ai_said in chain.stream(user_said):
                        print(ai_said, flush=True)
                else:
                    ai_said = chain.invoke(user_said)
                    print(ai_said)

            print("-"*20, "当前任务状态", "-"*20)
            print(self.root_content.all_todos())


In [25]:
wp = WritingProject()
wp.run()


👤:  帮我写一个500字小故事


{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '介绍故事的背景和主人公'}, {'标题名称': '冲突', '字数要求': 150, '内容摘要': '描述主人公遇到的困难和挑战'}, {'标题名称': '发展', '字数要求': 100, '内容摘要': '主人公如何应对困难，寻求解决方案'}, {'标题名称': '高潮', '字数要求': 100, '内容摘要': '故事达到高潮，展示主人公的勇敢和智慧'}, {'标题名称': '结局', '字数要求': 50, '内容摘要': '故事圆满结束，总结主人公的成长和收获'}], '大纲数量': 5}
-------------------- 当前任务状态 --------------------
[{'id': '20240505.110930.00014.605', 'type': 'paragraph'}]



👤:  


-------------------- 当前任务状态 --------------------
[{'id': '20240505.110930.00014.605', 'type': 'paragraph'}]



👤:  


-------------------- 当前任务状态 --------------------
[{'id': '20240505.110930.00014.605', 'type': 'paragraph'}]



👤:  quit


-------------------- quit


In [88]:
wp = WritingProject(streaming=True)
wp.run()


👤:  帮我写一个500字小故事


{}
{'类型': ''}
{'类型': 'out'}
{'类型': 'outlines'}
{'类型': 'outlines', '大纲列表': []}
{'类型': 'outlines', '大纲列表': [{}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}, {}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}, {'标题名称': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和


👤:  quit


-------------------- quit
